Careless responder feature engineering and analysis

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
import scipy as sp

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/final_data_complete.pkl'

In [ ]:
#read in csv containing data from all surveys
full_data = pd.read_pickle(data_path)
full_data = pd.DataFrame(data = full_data)
full_data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', full_data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', full_data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', full_data.info(), '\n')

In [ ]:
full_data['survey_type'].value_counts()

In [ ]:
#should be 71
len(full_data['wave_study_day'].unique())

In [ ]:
#create study date bins
full_data['wave_study_date_bin'] = pd.cut(full_data['wave_study_day'], 5)
full_data['wave_study_date_bin'].value_counts()

In [ ]:
full_data.head()

## CR Functions

In [ ]:
def context_homevsworking_CR(row, location_col, activity_col):
    
    #if at home should not be working
    if (row[location_col] == num_dic['location_home']) & (row[activity_col] == num_dic['activity_work']):
        context_homevsworking = 1
    else:
        context_homevsworking = 0

    return context_homevsworking

In [ ]:
def context_workvsactivities_CR(row, location_col, activity_col):
    
    #if at work should not be playing sports, household activities, civic duties
    if (row[location_col] == num_dic['location_work']) & \
    ((row[activity_col] == num_dic['activity_sports']) | (row[activity_col] == num_dic['activity_household']) | (row[activity_col] == num_dic['activity_civic'])):
        context_workvsactivities = 1
    else:
        context_workvsactivities = 0

    return context_workvsactivities

In [ ]:
def context_workvswork_CR(row, location_col, activity_col):
    
    #if at work should be working
    if (row[location_col] == num_dic['location_work']) & (row[activity_col] != num_dic['activity_work']):
        context_workvswork = 1
    else:
        context_workvswork = 0

    return context_workvswork

In [ ]:
def context_drivevsdrive_CR(row, location_col, activity_col):
    
    #if at vehicle should be driving/travel    
    if (row[location_col] == num_dic['location_vehicle']) & (row[activity_col] != num_dic['activity_drive']):
        context_drivevsdrive = 1
    else:
        context_drivevsdrive = 0

    return context_drivevsdrive

In [ ]:
def mean_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].dropna().values
    b = row.loc[param3:param4].dropna().values
    
    if (len(a) | len(b)) == 0.0:
        mean_diff = np.nan
    else:
        try:
            mean_diff = np.abs(np.mean(a) -  np.mean(b))
        except:
            print(row)
            mean_diff = np.nan
            
    return mean_diff

In [ ]:
def median_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        median_diff = np.nan
    else:
        try:
            median_diff = np.abs(np.median(a) -  np.median(b))
        except:
            print(row)
            median_diff = np.nan
        
    return median_diff

In [ ]:
def mode_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        mode_diff = np.nan
    else:
        try:
            mode_diff = np.abs(sp.stats.mode(a)[0][0] -  sp.stats.mode(b)[0][0])
        except:
            print(row)
            mode_diff = np.nan
    
    return mode_diff

In [ ]:
def longstring_CR(row):
    #create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

    groups = groupby(row)
    result = [(label, sum(1 for _ in group)) for label, group in groups]
    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    if max_string_length == 1:
        max_answer = 0
    return pd.Series((max_string_length, max_answer))

In [ ]:
def moments_CR(row):
    
    #auc
    auc = np.trapz(row.dropna())
    
    #std
    std = np.std(row.dropna())
    
    #skew
    if std == 0.0:
        skew = 0.0
    else:
        try:
            skew = sp.stats.skew(row.dropna())
        except:
            skew = np.nan
    
    #kurtosis
    if std == 0.0:
        kurt = -3.0
    else:
        try:
            kurt = sp.stats.kurtosis(row.dropna())
        except:
            kurt = np.nan
            
    return pd.Series((auc, std, skew, kurt))

In [ ]:
def moments_seeded_CR(row):
    
    #auc
    auc_seeded = np.trapz(np.append(row.dropna().values, 0.0))
    
    #std
    std_seeded = np.std(np.append(row.dropna().values, 0.0))

    #skew
    if std_seeded == 0.0:
        skew_seeded = 0.0
    else:
        try:
            skew_seeded = sp.stats.skew(np.append(row.dropna().values, 0.0))
        except:
            skew_seeded = np.nan
    
    #kurtosis
    if std_seeded == 0.0:
        kurt_seeded = -3.0
    else:
        try:
            kurt_seeded = sp.stats.kurtosis(np.append(row.dropna().values, 0.0))
        except:
            kurt_seeded = np.nan
            
    return pd.Series((auc_seeded, std_seeded, skew_seeded, kurt_seeded))

## CR feature creation

### notes for CR features for Engage surveys

Context question
- Semantic Antonyms
    - if context1 = home (0), then context2 ≠ work and work related (0)
    - if context1 = work (1), then context2 ≠ leisure sports (4), household activities (7), org/civic (11)
- Semantic Synonyms
    - if context1 = work (1), then context2 most likely work and work related (0)
    - If context1 = vehicle (4), then context2 most likely travel or commute (12)
- Internal consistency
    - if context1 = 5 (other) then should have a write in
    - if context2 = 13 (other) then should have a write in

Longstring
- All questions use same scale (1=not at all, 7=very much), but there are 5 different constructs assessed

Semantic consistency
- Internal consistency (within construct) should be greater than consistency across constructs

Semantic synonyms 
- not applicable 

Semantic antonyms
- Hindrance stressors should be negatively correlated with support 


In [ ]:
#split off completed engage and related columns
engage_only = full_data[(full_data['survey_type'] == 'engage_psycap') & (full_data['completed'] == 1.0)]

print(engage_only.shape)
engage_only['ParticipantID'].unique().shape

In [ ]:
%%time

num_dic = {'location_home': 0, 'location_work': 1, 'location_vehicle': 4,
           'activity_work': 0, 'activity_sports': 4, 'activity_household': 7, 'activity_civic': 11, 'activity_drive': 12}
location_col = 'engage_location'
activity_col = 'engage_activity'
engage_only['context_homevsworking'] = engage_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvsactivities'] = engage_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvswork'] = engage_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
engage_only['context_drivevsdrive'] = engage_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

param1 = 'support_mgt'
param2 = 'support_mgt'
param3 = 'hindrance_mgt'
param4 = 'hindrance_mgt'
engage_only['hinderance_vs_support_mean'] = engage_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
engage_only['hinderance_vs_support_median'] = engage_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
engage_only['hinderance_vs_support_mode'] = engage_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

param1 = 'engage_3'
param2 = 'engage_29'
engage_only[['longstring_count', 'longstring_answer']] = engage_only.apply(lambda row: longstring_CR(row.loc[param1:param2]), axis=1)
engage_only['longstring_mult'] = engage_only['longstring_count'] * engage_only['longstring_answer']

engage_only[['auc', 'std', 'skew', 'kurt']] = engage_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
engage_only[['auc_seeded', 'std_seeded', 'skew_seeded', 'kurt_seeded']] = engage_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

In [ ]:
engage_only.head()

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = full_data[(full_data['survey_type'] == 'psych_flex') & (full_data['completed'] == 1.0)]

print(psych_flex_only.shape)
psych_flex_only['ParticipantID'].unique().shape

In [ ]:
param1 = 'pf_03'
param2 = 'pf_15'

psych_flex_only[['longstring_count', 'longstring_answer']] = psych_flex_only.apply(lambda row: longstring_CR(row.loc[param1:param2]), axis=1)
psych_flex_only['longstring_mult'] = psych_flex_only['longstring_count'] * psych_flex_only['longstring_answer']

psych_flex_only[['auc', 'std', 'skew', 'kurt']] = psych_flex_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
psych_flex_only[['auc_seeded', 'std_seeded', 'skew_seeded', 'kurt_seeded']] = psych_flex_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

In [ ]:
psych_flex_only.head()

### notes for CR features for Jobs

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - IRB questions use same scale
    - dalal questions use same scale
- Consistency
    - irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
    - itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
    - dalal1-8 should be negatively correlated with dalal9-dalal16



In [ ]:
#split off completed job and related columns
jobs_atwork_only = full_data[(full_data['survey_type'] == 'job') & (full_data['work'] == 1)]

print(jobs_atwork_only.shape)
jobs_atwork_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
jobs_atwork_only['context_homevsworking'] = jobs_atwork_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvsactivities'] = jobs_atwork_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvswork'] = jobs_atwork_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_drivevsdrive'] = jobs_atwork_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
jobs_atwork_only['affect_posneg_mean'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['affect_posneg_median'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['affect_posneg_mode'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
jobs_atwork_only['nervous_anxiety_mean'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['nervous_anxiety_median'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['nervous_anxiety_mode'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
  
#irb1, irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
param1 = 'irbd1'
param2 ='irbd4'
param3 = 'irbd6'
param4 = 'irbd7'
jobs_atwork_only['irb_irb_mean'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['irb_irb_median'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['irb_irb_mode'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
param1 = 'itpd1'
param2 ='itpd3'
param3 = 'irbd6'
param4 = 'irbd7'
jobs_atwork_only['irb_itp_mean'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['irb_itp_median'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['irb_itp_mode'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#dalal1-8 should be negatively correlated with dalal9-dalal16
param1 = 'dalal1'
param2 ='dalal8'
param3 = 'dalal9'
param4 = 'dalal16'
jobs_atwork_only['dalal_posneg_mean'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['dalal_posneg_median'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
jobs_atwork_only['dalal_posneg_mode'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
jobs_atwork_only[['longstring_count_affect', 'longstring_answer_affect']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only['longstring_mult_affect'] = jobs_atwork_only['longstring_count_affect'] * jobs_atwork_only['longstring_answer_affect']
jobs_atwork_only[['auc_affect', 'std_affect', 'skew_affect', 'kurt_affect']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['auc_seeded_affect', 'std_seeded_affect', 'skew_seeded_affect', 'kurt_seeded_affect']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#irbd longstring
param1 = 'irbd1'
param2 = 'irbd7'
jobs_atwork_only[['longstring_count_irbd', 'longstring_answer_irbd']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only['longstring_mult_irbd'] = jobs_atwork_only['longstring_count_irbd'] * jobs_atwork_only['longstring_answer_irbd']
jobs_atwork_only[['auc_irbd', 'std_irbd', 'skew_irbd', 'kurt_irbd']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['auc_seeded_irbd', 'std_seeded_irbd', 'skew_seeded_irbd', 'kurt_seeded_irbd']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#dalal longstring
param1 = 'dalal1'
param2 = 'dalal16'
jobs_atwork_only[['longstring_count_dalal', 'longstring_answer_dalal']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only['longstring_mult_dalal'] = jobs_atwork_only['longstring_count_dalal'] * jobs_atwork_only['longstring_answer_dalal']
jobs_atwork_only[['auc_dalal', 'std_dalal', 'skew_dalal', 'kurt_dalal']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['auc_seeded_dalal', 'std_seeded_dalal', 'skew_seeded_dalal', 'kurt_seeded_dalal']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)


In [ ]:
jobs_atwork_only.head()

### explore CR across survey types

In [ ]:
engage_cols = ['context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive', 'hinderance_vs_support_mean',
       'hinderance_vs_support_median', 'hinderance_vs_support_mode',
       'longstring_count', 'longstring_answer', 'longstring_mult', 'auc',
       'std', 'skew', 'kurt', 'auc_seeded', 'std_seeded', 'skew_seeded',
       'kurt_seeded', 'time_to_complete']

psych_flex_cols = ['longstring_count',
       'longstring_answer', 'longstring_mult', 'auc', 'std', 'skew',
       'kurt', 'auc_seeded', 'std_seeded', 'skew_seeded', 'kurt_seeded', 'time_to_complete']

jobs_atwork_only_cols = ['context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive', 'affect_posneg_mean',
       'affect_posneg_median', 'affect_posneg_mode',
       'nervous_anxiety_mean', 'nervous_anxiety_median',
       'nervous_anxiety_mode', 'irb_irb_mean', 'irb_irb_median',
       'irb_irb_mode', 'irb_itp_mean', 'irb_itp_median', 'irb_itp_mode',
       'dalal_posneg_mean', 'dalal_posneg_median', 'dalal_posneg_mode',
       'longstring_count_affect', 'longstring_answer_affect',
       'longstring_mult_affect', 'auc_affect', 'std_affect',
       'skew_affect', 'kurt_affect', 'auc_seeded_affect',
       'std_seeded_affect', 'skew_seeded_affect', 'kurt_seeded_affect',
       'longstring_count_irbd', 'longstring_answer_irbd',
       'longstring_mult_irbd', 'auc_irbd', 'std_irbd', 'skew_irbd',
       'kurt_irbd', 'auc_seeded_irbd', 'std_seeded_irbd',
       'skew_seeded_irbd', 'kurt_seeded_irbd', 'longstring_count_dalal',
       'longstring_answer_dalal', 'longstring_mult_dalal', 'auc_dalal',
       'std_dalal', 'skew_dalal', 'kurt_dalal', 'auc_seeded_dalal',
       'std_seeded_dalal', 'skew_seeded_dalal', 'kurt_seeded_dalal', 'time_to_complete']

In [ ]:
engage_only.dropna(subset=['hinderance_vs_support_mean'], inplace=True)
engage_only.shape

In [ ]:
jobs_atwork_only.dropna(axis=0, inplace=True)
jobs_atwork_only.shape

In [ ]:
jobs_atwork_only[jobs_atwork_only['irb_irb_mean'].isnull()].head()

In [ ]:
jobs_CRs = jobs_atwork_only[jobs_atwork_only_cols].dropna(axis=0)

In [ ]:
jobs_CRs.corr()

In [ ]:
for feature in jobs_atwork_only_cols:
    sns.jointplot(jobs_CRs['irb_irb_mean'], jobs_CRs[feature], kind='hex')
    plt.show()

In [ ]:
jobs_atwork_only[jobs_atwork_only_cols].corr()

In [ ]:
CR_data = full_data[general_cols]
CR_data = pd.merge(CR_data, engage_only[engage_cols], on=['survey_id'], how='left', suffixes=('_demog', '_engage'))
CR_data = pd.merge(CR_data, psych_flex_only[psych_flex_cols], on=['survey_id'], how='left', suffixes=('_demog', '_psy_flex'))
CR_data = pd.merge(CR_data, jobs_atwork_only[jobs_atwork_only_cols], on=['survey_id'], how='left', suffixes=('_demog', 'jobs'))
CR_data.head()

In [ ]:
sns.pairplot(engage_only[potential_features_cont], diag_kind='kde')

In [ ]:
engage_only_features = engage_only[['MitreID', 
                                    'hinderance_vs_support_SAD', 
                                    'longstring_count',
                                    'longstring_mult', 
                                    'skew_seeded',
                                    'kurt_seeded',
                                    'time_to_complete']]
engage_only_features.set_index('MitreID', inplace=True)
engage_only_features.head()

In [ ]:
# center and scale the data
scaler = StandardScaler()

engage_survey_features_scaled = scaler.fit_transform(engage_only_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(engage_survey_features_scaled)
    scores.append(silhouette_score(engage_survey_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
engage_km_survey = KMeans(n_clusters=2,random_state=1234)
engage_km_survey.fit(engage_survey_features_scaled)
print(silhouette_score(engage_survey_features_scaled, engage_km_survey.labels_))

engage_only_features['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only_features_cont['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]

engage_only['kmeans_scaled_survey'].value_counts()

In [ ]:
sns.pairplot(engage_only_features_cont, hue = 'kmeans_scaled_survey', diag_kind='kde')

In [ ]:
for feature in potential_features:
    sns.barplot(x='kmeans_scaled_survey', y=feature, data=engage_only)
    plt.show()

In [ ]:
for feature in potential_features:
    sns.barplot(x='longest_string_count', y=feature, hue='kmeans_scaled_survey', data=engage_only)
    plt.show()

In [ ]:
sns.countplot(x='wave_study_date_bin', hue='kmeans_scaled_survey', data=engage_only)

In [ ]:
participants = engage_only['MitreID'].unique()

for part in participants:
    data_part = engage_only[engage_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()

In [ ]:
#add cluster 1 % to final data
participants = data['MitreID'].unique()

for part in participants:
    if part in engage_only_features.index:
        perc = engage_only_features[(engage_only_features.index == part) & (engage_only_features['kmeans_scaled_survey'] == 1)].shape[0] / engage_only_features[engage_only_features.index == part].shape[0]

        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = perc
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = perc
    
    else:
        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = np.nan
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = np.nan

In [ ]:
sns.barplot(x='MitreID', y='engage_CR_perc', data=engage_only)

## Features and clustering on PF surveys

In [ ]:
psych_flex_only.dropna(subset=['longest_string_mult'], inplace=True)
psych_flex_only.shape

In [ ]:
#viz relationship and correlation across possible features
potential_features = ['longest_string_count', 'longest_string_answer', 'longest_string_mult',
                      'auc', 'std', 'skew', 'kurt', 'auc_seeded', 'std_seeded', 'skew_seeded', 'kurt_seeded', 
                      'time_to_complete']

psych_flex_only_features_potential = psych_flex_only[potential_features]
psych_flex_only_features_potential.corr()

In [ ]:
for feature in potential_features:
    sns.jointplot(psych_flex_only_features_potential['longest_string_count'], psych_flex_only_features_potential[feature], kind='hex')
    plt.show()

In [ ]:
sns.pairplot(psych_flex_only_features_potential, diag_kind='kde')

In [ ]:
psych_flex_only_features = psych_flex_only[['MitreID', 
                                            'longest_string_count',
                                            'longest_string_mult',
                                            'time_to_complete']]
psych_flex_only_features.set_index('MitreID', inplace=True)
psych_flex_only_features.head()

In [ ]:
# center and scale the data
scaler = StandardScaler()

PF_survey_features_scaled = scaler.fit_transform(psych_flex_only_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(PF_survey_features_scaled)
    scores.append(silhouette_score(PF_survey_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
PF_km_survey = KMeans(n_clusters=3,random_state=1234)
PF_km_survey.fit(PF_survey_features_scaled)
print(silhouette_score(PF_survey_features_scaled, PF_km_survey.labels_))

psych_flex_only_features_potential['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]
psych_flex_only_features['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]
psych_flex_only['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]

psych_flex_only['kmeans_scaled_survey'].value_counts()

In [ ]:
for feature in potential_features:
    sns.barplot(x='kmeans_scaled_survey', y=feature, data=psych_flex_only)
    plt.show()

In [ ]:
sns.pairplot(psych_flex_only_features_potential, hue = 'kmeans_scaled_survey', diag_kind='kde')

In [ ]:
#add cluster 1 % to final data
participants = data['MitreID'].unique()

for part in participants:
    if part in psych_flex_only_features.index:
        perc = psych_flex_only_features[(psych_flex_only_features.index == part) & (psych_flex_only_features['kmeans_scaled_survey'] == 0)].shape[0] / psych_flex_only_features[psych_flex_only_features.index == part].shape[0]

        data.loc[data['MitreID'] == part, 'pf_CR_perc'] = perc
        psych_flex_only.loc[psych_flex_only['MitreID'] == part, 'pf_CR_perc'] = perc
    
    else:
        data.loc[data['MitreID'] == part, 'pf_CR_perc'] = np.nan
        psych_flex_only.loc[psych_flex_only['MitreID'] == part, 'pf_CR_perc'] = np.nan

In [ ]:
sns.countplot(x='wave_study_date_bin', hue='kmeans_scaled_survey', data=psych_flex_only)

In [ ]:
sns.barplot(x='MitreID', y='pf_CR_perc', data=psych_flex_only)

In [ ]:
sns.jointplot(x='pf_CR_perc', y='engage_CR_perc', data=data, kind='reg')

In [ ]:
participants = psych_flex_only['MitreID'].unique()

for part in participants:
    print(part)
    data_part = psych_flex_only[psych_flex_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()
    data_part = engage_only[engage_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()

In [ ]:
job_only.columns.values

In [ ]:
job_only.isna().sum()

In [ ]:
job_only.dropna(subset=['irb_itp'], inplace=True)
job_only.shape

In [ ]:
#viz relationship and correlation across possible features
potential_features = ['affect_posneg',
       'nervous_anxiety', 'irb_irb', 'irb_itp', 'dalal_posneg', 'time_to_complete',
       'longest_string_count_affect', 'longest_string_answer_affect',
       'longest_string_mult_affect', 'longest_string_count_irb',
       'longest_string_answer_irb', 'longest_string_mult_irb',
       'longest_string_count_dalal', 'longest_string_answer_dalal',
       'longest_string_mult_dalal', 'context_homevsworking', 'context_workvsactivities',
       'context_workvswork', 'context_drivevsdrive']

job_only_features_potential = job_only[potential_features]
job_only_features_potential.corr()

In [ ]:
job_only.head()

In [ ]:
#raw and seeded auc, std, skew and kurtosis (seeded is trying to deal with 0 skew of all same answer vs normally distributed answers)
auc = []
std = []
skew = []
kurt = []

auc_seeded = []
std_seeded = []
skew_seeded = []
kurt_seeded = []

for index, row in psych_flex_only.iterrows():

    #auc
    num_auc = np.trapz(row.loc['pf_03':'pf_15'].dropna())
    auc.append(num_auc)
    num_auc_seeded = np.trapz(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
    auc_seeded.append(num_auc_seeded)
    
    #std
    num_std = np.std(row.loc['pf_03':'pf_15'].dropna())
    std.append(num_std)
    num_std_seeded = np.std(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
    std_seeded.append(num_std_seeded)
    
    #skew
    if num_std == 0.0:
        num_skew = 0.0
    else:
        try:
            num_skew = sp.stats.skew(row.loc['pf_03':'pf_15'].dropna())
        except:
            num_skew = np.nan
    skew.append(num_skew)
    num_skew_seeded = sp.stats.skew(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
    skew_seeded.append(num_skew_seeded)
    
    #kurtosis
    if num_std == 0.0:
        num_kurt = -3.0
    else:
        try:
            num_kurt = sp.stats.kurtosis(row.loc['pf_03':'pf_15'].dropna())
        except:
            num_kurt = np.nan
    kurt.append(num_kurt)
    try:
        num_kurt_seeded = sp.stats.kurtosis(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
    except:
        num_kurt_seeded = np.nan
    kurt_seeded.append(num_kurt_seeded)

psych_flex_only['auc'] = auc 
psych_flex_only['std'] = std    
psych_flex_only['skew'] = skew
psych_flex_only['kurt'] = kurt

psych_flex_only['auc_seeded'] = auc_seeded 
psych_flex_only['std_seeded'] = std_seeded    
psych_flex_only['skew_seeded'] = skew_seeded
psych_flex_only['kurt_seeded'] = kurt_seeded

In [ ]:
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings_affect = []
max_answers_affect = []
longstring_mult_affect = []

max_strings_irb = []
max_answers_irb = []
longstring_mult_irb = []

max_strings_dalal = []
max_answers_dalal = []
longstring_mult_dalal = []

for index, row in job_only.iterrows():
    
    #affect/anxiety/stress longstring
    groups_affect = groupby(row[['pand1', 'pand2', 'pand3', 'pand4', 'pand5', 'pand6', 'pand7', 'pand8', 'pand9', 'pand10', 'anxiety', 'stress']])
    result_affect = [(label, sum(1 for _ in group)) for label, group in groups_affect]
    max_pair_affect = max(result_affect, key=lambda x:x[1])
    max_string_length_affect = max_pair_affect[1]
    max_answer_affect = max_pair_affect[0]
    mult_affect = max_string_length_affect*max_answer_affect
    max_strings_affect.append(max_string_length_affect)
    max_answers_affect.append(max_answer_affect)
    longstring_mult_affect.append(mult_affect)
    
    #dalal longstring
    groups_irb = groupby(row['irbd1':'irbd7'])
    result_irb = [(label, sum(1 for _ in group)) for label, group in groups_irb]
    max_pair_irb = max(result_irb, key=lambda x:x[1])
    max_string_length_irb = max_pair_irb[1]
    max_answer_irb = max_pair_irb[0]
    mult_irb = max_string_length_irb*max_answer_irb
    max_strings_irb.append(max_string_length_irb)
    max_answers_irb.append(max_answer_irb)
    longstring_mult_irb.append(mult_irb)
    
    #dalal longstring
    groups_dalal = groupby(row['dalal1':'dalal16'])
    result_dalal = [(label, sum(1 for _ in group)) for label, group in groups_dalal]
    max_pair_dalal = max(result_dalal, key=lambda x:x[1])
    max_string_length_dalal = max_pair_dalal[1]
    max_answer_dalal = max_pair_dalal[0]
    mult_irb = max_string_length_dalal*max_answer_dalal
    max_strings_dalal.append(max_string_length_dalal)
    max_answers_dalal.append(max_answer_dalal)
    longstring_mult_dalal.append(mult_irb)
    
job_only['longest_string_count_affect'] = max_strings_affect
job_only['longest_string_answer_affect'] = max_answers_affect
job_only['longest_string_mult_affect'] = longstring_mult_affect

job_only['longest_string_count_irb'] = max_strings_irb
job_only['longest_string_answer_irb'] = max_answers_irb
job_only['longest_string_mult_irb'] = longstring_mult_irb

job_only['longest_string_count_dalal'] = max_strings_dalal
job_only['longest_string_answer_dalal'] = max_answers_dalal
job_only['longest_string_mult_dalal'] = longstring_mult_dalal